# Imports and installations

In [ ]:
!pip install cartopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

# Combine Plot


We have plotted anomalies of **500 hPa geopotential height and wind**, along with **vertically averaged potential vorticity (500–200 hPa)**.  

- Geopotential height was calculated by dividing the 500 hPa geopotential by `9.8 m/s²`.  
- Anomalies were calculated in:  
  - `anomaly_ght_and_wind_at_500hpa_calculation.ipynb`  
  - `anomaly_pv_vertically_averaged_calculation.ipynb`
  


In [ ]:


projection = ccrs.PlateCarree()
fig, axes = plt.subplots(nrows=2, ncols=2, subplot_kw={'projection':projection}, figsize=(15, 30))

# Geopotential height (levels)
levels_ght = np.arange(-40, 40 + 1, 5)

# ---- Pre-monsoon (MAM) GHT + Wind ----
ax = axes[0,0]
# Load data
ght = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/Scripts-ll/ght_wind/data/ght_events_anom-mam.nc')
wind = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/Scripts-ll/ght_wind/data/wind_500_events_anom-mam.nc')

# Mean anomalies
ght_anom = ght.mean(dim='event_no', skipna=True)
wind_anom = wind.mean(dim='event_no', skipna=True)

# Lat/lon arrays
latitude_ght = ght_anom['latitude'].values
longitude_ght = ght_anom['longitude'].values
wind_lat = wind_anom['latitude'].values
wind_lon = wind_anom['longitude'].values

# Wind components (subsampled)
u = wind_anom.u
v = wind_anom.v
stride = 12
subsampled_u = u[::stride, ::stride]
subsampled_v = v[::stride, ::stride]
wind_lat_subsampled = wind_lat[::stride]
wind_lon_subsampled = wind_lon[::stride]

# Contour + quiver
contourf = ax.contourf(longitude_ght, latitude_ght, ght_anom.z/9.8,
                       levels=levels_ght, cmap='seismic', extend='both', transform=projection)
quiver = ax.quiver(wind_lon_subsampled, wind_lat_subsampled, subsampled_u, subsampled_v,
                   transform=ccrs.PlateCarree(), color='black')
quiver_key = ax.quiverkey(quiver, 0.89, 1.04, 3, r'3 ${ms^{-1}}$', labelpos='E', coordinates='axes')

# Map features
ax.gridlines(draw_labels=['left','bottom'], visible=False)
ax.coastlines()
ax.set_title("(a) Pre-monsoon", fontsize=9)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

# ---- Monsoon (JJAS) GHT + Wind ----
ax = axes[0,1]
# Load data
ght = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/Scripts-ll/ght_wind/data/ght_events_anom-jjas.nc')
wind = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/Scripts-ll/ght_wind/data/wind_500_events_anom-jjas.nc')

# Mean anomalies
ght_anom = ght.mean(dim='event_no', skipna=True)
wind_anom = wind.mean(dim='event_no', skipna=True)

# Lat/lon arrays
latitude_ght = ght_anom['latitude'].values
longitude_ght = ght_anom['longitude'].values
wind_lat = wind_anom['latitude'].values
wind_lon = wind_anom['longitude'].values

# Wind components (subsampled)
u = wind_anom.u
v = wind_anom.v
stride = 12
subsampled_u = u[::stride, ::stride]
subsampled_v = v[::stride, ::stride]
wind_lat_subsampled = wind_lat[::stride]
wind_lon_subsampled = wind_lon[::stride]

# Contour + quiver
contourf = ax.contourf(longitude_ght, latitude_ght, ght_anom.z/9.8,
                       levels=levels_ght, cmap='seismic', extend='both', transform=projection)
quiver = ax.quiver(wind_lon_subsampled, wind_lat_subsampled, subsampled_u, subsampled_v,
                   transform=ccrs.PlateCarree(), color='black')
quiver_key = ax.quiverkey(quiver, 0.89, 1.04, 3, r'3 ${ms^{-1}}$', labelpos='E', coordinates='axes')

# Map features
ax.gridlines(draw_labels=['bottom'], visible=False)
ax.coastlines()
ax.set_title("(b) Monsoon", fontsize=9)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))

# Shared colorbar (GHT)
cbar = plt.colorbar(contourf, ax=axes[0,:], orientation='horizontal', pad=0.01,
                    fraction=0.02, aspect=35, label="${m}$")
cbar.set_ticks(levels_ght[::2])
cbar.ax.set_position([0.21, 0.232, 0.6, 0.025])
fig.text(0.5, 0.365, "Composite of 500 hPa Geopotential Height (shaded) and Wind (vectors) Anomalies",
         ha='center', fontsize=10, fontweight='bold')

# PV anomaly levels
levels_pv = np.arange(-2.5e-7, 2.5e-7 + 0.05e-7, 0.25e-7)

# ---- Pre-monsoon (MAM) PV ----
ax = axes[1,0]
pv = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/pv/data/pv_level_avg_anom-mam.nc')
pv = pv.mean(dim='event_no', skipna=True)

# Lat/lon arrays
latitude = pv['latitude'].values
longitude = pv['longitude'].values

# Contour plot
contourf = ax.contourf(longitude, latitude, pv.pv, levels=levels_pv,
                       cmap='seismic', extend='both', transform=projection)

# Map features
ax.coastlines()
ax.set_title("(c) Pre-monsoon", fontsize=9)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))
ax.gridlines(draw_labels=['left','bottom'], visible=False)

# ---- Monsoon (JJAS) PV ----
ax = axes[1,1]
pv = xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/pv/data/pv_level_avg_anom-jjas.nc')
pv = pv.mean(dim='event_no', skipna=True)

# Lat/lon arrays
latitude = pv['latitude'].values
longitude = pv['longitude'].values

# Contour plot
contourf = ax.contourf(longitude, latitude, pv.pv, levels=levels_pv,
                       cmap='seismic', extend='both', transform=projection)

# Map features
ax.coastlines()
ax.set_title("(d) Monsoon", fontsize=9)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.add_feature(cf.BORDERS.with_scale('10m'))
ax.gridlines(draw_labels=['bottom'], visible=False)

# Shared colorbar (PV)
cbar = plt.colorbar(contourf, ax=axes[1,:], orientation='horizontal', pad=0.01,
                    fraction=0.02, aspect=35, label="${K}{m}^{2}{kg}^{-1} s^{-1}$")
cbar.set_ticks(levels_pv[::2])
cbar.ax.set_position([0.21, 0.075, 0.6, 0.025])
fig.text(0.5, 0.225, "Composite of 500–200 hPa Vertically Averaged PV Anomalies",
         ha='center', fontsize=10, fontweight='bold')

# Adjust subplot spacing
plt.subplots_adjust(wspace=0.05, hspace=-0.75, top=0.9)

plt.show()

# Save the figure
fig.savefig("/content/drive/MyDrive/AP_HW/Scripts-ll/Manuscript codes/All_Figures/Fig_5_composite_of_ght_wind_pv.jpg",
            dpi=300, format="jpg", bbox_inches="tight")
